# Steam Review Sentiment Analysis (PySpark + GCP)
This notebook loads and processes Steam game reviews using PySpark on Google Cloud Dataproc. The raw dataset is stored in GCS (~47GB) and loaded with a defined schema for performance and type consistency.

In [ ]:
spark

## 1. Imports & Google Cloud Storage (GCS) Setup

In [ ]:
from pyspark.sql.functions import col, isnan, count, udf, length, when
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.cloud import storage
from pyspark.sql.functions import desc

In [ ]:
sc.setLogLevel("ERROR")

In [ ]:
# path to files (REPLACE WITH YOUR OWN GCS BUCKET WHEN RUNNING THE NOTEBOOK)
bucket = 'gs://Whatever-your-bucket name is'
landing_folder = f"{bucket}/landing/all_reviews.csv"
cleaned_folder = f"{bucket}/cleaned/"

## 2. Define Schema & Load In Raw Data

In [ ]:
# preset schema for predictors since inferSchema was only giving string
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType
steamSchema = StructType([
    StructField("recommendationid", LongType(), True),              # Large integer (148919893)
    StructField("appid", IntegerType(), True),                      # Small integer (10)
    StructField("game", StringType(), True),                        # Text ("Counter-Strike")
    StructField("author_steamid", LongType(), True),                # Large integer (76561199036724879)
    StructField("author_num_games_owned", IntegerType(), True),     # Small integer (0)
    StructField("author_num_reviews", IntegerType(), True),         # Small integer (3)
    StructField("author_playtime_forever", IntegerType(), True),    # Integer (197)
    StructField("author_playtime_last_two_weeks", IntegerType(), True),  # Integer (197)
    StructField("author_playtime_at_review", IntegerType(), True),  # Integer (197)
    StructField("author_last_played", LongType(), True),            # Unix timestamp (1698336369)
    StructField("language", StringType(), True),                    # Text ("russian")
    StructField("review", StringType(), True),                      # Text ("старость")
    StructField("timestamp_created", LongType(), True),             # Unix timestamp (1698336397)
    StructField("timestamp_updated", LongType(), True),             # Unix timestamp (1698336397)
    StructField("voted_up", IntegerType(), True),                   # Small integer (1 as true)
    StructField("votes_up", IntegerType(), True),                   # Small integer (0)
    StructField("votes_funny", IntegerType(), True),                # Small integer (0)
    StructField("weighted_vote_score", FloatType(), True),          # Float (0.0)
    StructField("comment_count", IntegerType(), True),              # Small integer (0)
    StructField("steam_purchase", IntegerType(), True),             # Small integer (1 as true)
    StructField("received_for_free", IntegerType(), True),          # Small integer (0 as false)
    StructField("written_during_early_access", IntegerType(), True),  # Small integer (0 as false)
    StructField("hidden_in_steam_china", IntegerType(), True),       # Small integer (1 as true)
    StructField("steam_china_location", StringType(), True)
])

In [ ]:
# Load raw review data from GCS bucket with defined schema
sdf = spark.read.csv(landing_folder, header=True, schema=steamSchema) #loading in data in set schema to DataFrame

In [ ]:
sdf.show(1, vertical=True)

## 3. Data Cleaning (Removing nulls and Imputing Values)

In [ ]:
# Drop columns that provide little to no value
sdf = sdf.drop('recommendationid', 'appid', 'author_steamid','steam_china_location', 'timestamp_updated', 'hidden_in_steam_china') # drop non numeric/minimally useful columns 

In [ ]:
sdf.show(1, vertical=True)

In [ ]:
column_list = [
    "author_num_games_owned",
    "author_num_reviews",
    "author_playtime_forever",
    "author_playtime_last_two_weeks",
    "author_playtime_at_review",
    "author_last_played",
    "game",
    "review",
    "language",
    "timestamp_created",
    "voted_up",
    "votes_up",
    "votes_funny",
    "weighted_vote_score",
    "comment_count",
    "steam_purchase",
    "received_for_free",
    "written_during_early_access"
]

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
median_apf = sdf.approxQuantile("author_playtime_forever", [0.5], 0.01)[0] #fill na with median
sdf = sdf.fillna({'author_playtime_forever': median_apf})

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
median_apfltw = sdf.approxQuantile("author_playtime_last_two_weeks", [0.5], 0.01)[0] #fill na with median
sdf = sdf.fillna({'author_playtime_last_two_weeks': median_apfltw})

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
sdf = sdf.fillna({'author_last_played': 0}) #fill in null with 0

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
sdf = sdf.fillna({'voted_up': 0, 'votes_up': 0, 'votes_funny': 0, 'weighted_vote_score': 0, 'comment_count':0}) #fill in null with 0

In [ ]:
sdf = sdf.fillna({'timestamp_created': 0}) # Fill nulls with 0

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
mode_sp = sdf.groupBy("steam_purchase").count().orderBy(desc("count")).first()[0] # Get mode for steam_purchase

In [ ]:
mode_rff = sdf.groupBy("received_for_free").count().orderBy(desc("count")).first()[0] # Get mode for recieve_for_free

In [ ]:
mode_wdea = sdf.groupBy("written_during_early_access").count().orderBy(desc("count")).first()[0] # Get mode for WDEA

In [ ]:
# Fill nulls with mode
sdf = sdf.fillna({"steam_purchase": mode_sp, 'received_for_free': mode_rff, 'written_during_early_access': mode_wdea})

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
# Remove rows where game value is missing
sdf = sdf.dropna(subset=["game"])

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
# Display nulls in the review column
sdf.select(count(when(col("review").isNull(), "review")).alias("review_nulls")).show()

In [ ]:
# Check for nulls & NaN since they differ in pyspark
from pyspark.sql.functions import isnan, count, when, col
sdf.select(
    count(when(isnan('review'), 'review')).alias('review_nan'),
    count(when(col('review').isNull(), 'review')).alias('review_null')).show()

In [ ]:
# Replace review column with one where rows with NaN values are dropped
sdf = sdf.withColumn('review', when (isnan('review'), None).otherwise(col('review'))).dropna(subset=['review'])

In [ ]:
# Filter languages to english for text processing
sdf = sdf.filter(sdf['language'] == 'english')

In [ ]:
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in column_list]).show(vertical=True) #shows nulls for each column

In [ ]:
# Count number of records
sdf.count()

In [ ]:
sdf.rdd.getNumPartitions()

## 4. Save Cleaned Dataset

In [ ]:
cleaned_filename = f"{cleaned_folder}/cleaned_steam_reviews_data.parquet" # save to google cloud clean folder as parquet
sdf.write.parquet(cleaned_filename, mode='overwrite')